In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = "/content/drive/My Drive/NICOS_NN/SOLSTICIO2_0"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import xarray as xr
import os

In [ ]:
ds = xr.open_dataset(f'{folder_path}/MERRA_nv.nc')
layers = range(5,72)
time_indexes= range(0,1)
lat_indexes = range(35,326,2)
lon_indexes = range(0,576,4)

In [ ]:
ds

<xarray.Dataset> Size: 299MB
Dimensions:  (time: 1, lon: 576, lat: 361, lev: 72)
Coordinates:
  * time     (time) datetime64[ns] 8B 2024-06-20
  * lon      (lon) float64 5kB -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5 90.0
  * lev      (lev) float64 576B 1.0 2.0 3.0 4.0 5.0 ... 68.0 69.0 70.0 71.0 72.0
Data variables:
    DELP     (time, lev, lat, lon) float32 60MB ...
    H        (time, lev, lat, lon) float32 60MB ...
    PL       (time, lev, lat, lon) float32 60MB ...
    QV       (time, lev, lat, lon) float32 60MB ...
    T        (time, lev, lat, lon) float32 60MB ...
Attributes: (12/32)
    CDI:                               Climate Data Interface version 1.9.8 (...
    Conventions:                       CF-1
    History:                           Original file generated: Tue Jul 16 23...
    Comment:                           GMAO filename: d5124_m2_jan10.inst3_3d...
    Filename:                          MERRA2_400.inst3_3d_asm_Nv.20240620.nc4
    Institution:                       NASA Global Modeling and Assimilation ...
    ...                                ...
    RangeBeginningDate:                2024-06-20
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2024-06-20
    RangeEndingTime:                   21:00:00.000000
    history_L34RS:                     'Created by L34RS v1.4.4 @ NASA GES DI...
    CDO:                               Climate Data Operators version 1.9.8 (...

In [ ]:
os.makedirs(folder_path, exist_ok=True)
output_file_path = os.path.join(folder_path, 'merradf_output.txt')
with open(output_file_path, 'w') as file:
    for time_index in time_indexes:
        for lat_index in lat_indexes:
            for lon_index in lon_indexes:
                for layer in layers:
                    pl_value = ds.PL.isel(time=time_index, lev=layer, lat=lat_index, lon=lon_index).values/100
                    qv_value = ds.QV.isel(time=time_index, lev=layer, lat=lat_index, lon=lon_index).values
                    t_value   = ds.T.isel(time=time_index, lev=layer, lat=lat_index, lon=lon_index).values
                    delp_value = ds.DELP.isel(time=time_index, lev=layer, lat=lat_index, lon=lon_index).values/100

#                    time_value = ds.time.isel(time=time_index).values
                    lat_value = ds.lat.isel(lat=lat_index).values
                    lon_value = ds.lon.isel(lon=lon_index).values

                    if np.isscalar(t_value):
                        continue

                    #specific humidity (QV) to WaterVaporMixingRation (WVMR), and then to mass/mass fraction of WVMR (H2O)
                    #Conversion h2o is made from dividing dry air molecular mass (28,964 g/mol)  and water vapor molecular mass (18,015 g/mol)
                    wvmr = (qv_value)/(1.0 - qv_value)
                    h2o_value = (28.964/18.015)*wvmr

                    #Interpolation for PL
                    pl_value = pl_value + 0.5*delp_value

                    #Interpolation for temperature
                    if t_value.size > 1:
                        t_value = (t_value[:-1] + t_value[1:]) / 2
                        t_value = np.append(t_value, t_value[-1])

#                    file.write(f"{pl_value}, {h2o_value}, {t_value}, {delp_value}\n")
                    file.write(f"{lat_value}, {lon_value}, {pl_value}, {h2o_value}, {t_value}, {delp_value}\n")

#                    print(f"{pl_value} [mbar], {h2o_value} [vmr], {t_value} [K], {delp_value} [mbar]")
                    print(f"{lat_value}, {lon_value}, {pl_value} [mbar], {h2o_value} [vmr], {t_value} [K], {delp_value} [mbar]")

Se han truncado las últimas 5000 líneas del flujo de salida.
23.730403900146484 [mbar], 5.60683338335366e-06 [vmr], 196.18222045898438 [K], 3.9388012886047363 [mbar]
28.36780548095703 [mbar], 5.445353508548578e-06 [vmr], 196.87997436523438 [K], 4.637399673461914 [mbar]
33.81000518798828 [mbar], 5.301627879816806e-06 [vmr], 197.4786834716797 [K], 5.442202091217041 [mbar]
40.17540740966797 [mbar], 5.1841734602930956e-06 [vmr], 198.42245483398438 [K], 6.365402698516846 [mbar]
47.64390563964844 [mbar], 5.067061465524603e-06 [vmr], 199.55181884765625 [K], 7.468501091003418 [mbar]
56.387908935546875 [mbar], 4.88111663798918e-06 [vmr], 200.71348571777344 [K], 8.743999481201172 [mbar]
66.60340881347656 [mbar], 4.663332219934091e-06 [vmr], 201.52914428710938 [K], 10.215502738952637 [mbar]
78.51231384277344 [mbar], 4.67932159153861e-06 [vmr], 202.94680786132812 [K], 11.90890121459961 [mbar]
92.36571502685547 [mbar], 4.8911924750427715e-06 [vmr], 204.98666381835938 [K], 13.853407859802246 [mbar]


KeyboardInterrupt: 